In [ ]:
!pip3 install dgl
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as  F

import dgl
from transformers import pipeline
import numpy as np
import pickle
from tqdm import tqdm
import sys
import os
import pandas as pd
import time

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
from collections import defaultdict, Counter

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/5fold/train_0.tsv', delimiter='\t')
df_test = pd.read_csv('/content/drive/MyDrive/5fold/test_0.tsv', delimiter='\t')
df_dev = pd.read_csv('/content/drive/MyDrive/5fold/dev.tsv', delimiter='\t')

In [ ]:
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sample_df = df_dev[:100]

In [ ]:
sample_df['supporting_score'] = sample_df.progress_apply(lambda x : classifier(x['claim_text'], x['evidence'])['scores'][0], axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-28-2a9d014839bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['supporting_score'] = sample_df.progress_apply(lambda x : classifier(x['claim_text'], x['evidence'])['scores'][0], axis=1)


In [ ]:
sample_df['is_supported'] = sample_df['supporting_score'].apply(lambda x: int(x >= 0.5))

<ipython-input-31-b87d18765769>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['is_supported'] = sample_df['supporting_score'].apply(lambda x: int(x >= 0.5))


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
def extract_nouns(text):
  doc = nlp(text)
  return [token.lemma_ for token in doc if token.pos_ == "NOUN"]

In [ ]:
claim_ids, claim_texts = {}, []
evidence_source_ids, evidence_sources = {}, []
noun_to_claim_ids = defaultdict(set)
evidence_source_to_supported_claim_ids = defaultdict(set)
evidence_source_to_not_supported_claim_ids = defaultdict(set)

for _, row in tqdm(sample_df.iterrows()):
  if row.claim_id not in claim_ids:
    claim_ids[row.claim_id] = len(claim_ids)
    claim_texts.append(row.claim_text)
    for noun in extract_nouns(row.claim_text):
      noun_to_claim_ids[noun].add(claim_ids[row.claim_id])
  claim_id = claim_ids[row.claim_id]

  if row.evidence_source not in evidence_source_ids:
    evidence_source_ids[row.evidence_source] = len(evidence_source_ids)
    evidence_sources.append(row.evidence_source)
  evidence_source_id = evidence_source_ids[row.evidence_source]

  if row.is_supported:
    evidence_source_to_supported_claim_ids[evidence_source_id].add(claim_id)
  else:
    evidence_source_to_not_supported_claim_ids[evidence_source_id].add(claim_id)


100it [00:00, 334.91it/s]


In [ ]:
common_np_edges = set()
for claim_ids in noun_to_claim_ids.values():
  claim_ids = list(claim_ids)
  for i in range(len(claim_ids)):
    for j in range(i + 1, len(claim_ids)):
      common_np_edges.add((claim_ids[i], claim_ids[j]))
      common_np_edges.add((claim_ids[j], claim_ids[i]))

In [ ]:
sup_sup_edges = set()
for claim_ids in evidence_source_to_supported_claim_ids.values():
  claim_ids = list(claim_ids)
  for i in range(len(claim_ids)):
    for j in range(i + 1, len(claim_ids)):
      sup_sup_edges.add((claim_ids[i], claim_ids[j]))
      sup_sup_edges.add((claim_ids[j], claim_ids[i]))

In [ ]:
nosup_nosup_edges = set()
for claim_ids in evidence_source_to_not_supported_claim_ids.values():
  claim_ids = list(claim_ids)
  for i in range(len(claim_ids)):
    for j in range(i + 1, len(claim_ids)):
      nosup_nosup_edges.add((claim_ids[i], claim_ids[j]))
      nosup_nosup_edges.add((claim_ids[j], claim_ids[i]))

In [ ]:
sup_nosup_edges = set()
evidence_sources = set(evidence_source_to_not_supported_claim_ids).intersection(evidence_source_to_supported_claim_ids)
for evidence_source in evidence_sources:
  supported_claim_ids = list(evidence_source_to_supported_claim_ids[evidence_source])
  not_supported_claim_ids = list(evidence_source_to_not_supported_claim_ids[evidence_source])
  for i in range(len(supported_claim_ids)):
    for j in range(len(not_supported_claim_ids)):
      sup_nosup_edges.add((supported_claim_ids[i], not_supported_claim_ids[j]))
      sup_nosup_edges.add((not_supported_claim_ids[j], supported_claim_ids[i]))

In [ ]:
common_np_edges = np.array([list(edge) for edge in common_np_edges])
sup_sup_edges = np.array([list(edge) for edge in sup_sup_edges])
nosup_nosup_edges = np.array([list(edge) for edge in nosup_nosup_edges])
sup_nosup_edges = np.array([list(edge) for edge in sup_nosup_edges])

In [ ]:


graph = dgl.heterograph({
    ('claim', 'noun', 'claim') : (common_np_edges[:, 0], common_np_edges[:, 1]),
    ('claim', 'sup_sup', 'claim') : (sup_sup_edges[:, 0], sup_sup_edges[:, 1]),
    ('claim', 'nosup_nosup', 'claim') : (nosup_nosup_edges[:, 0], nosup_nosup_edges[:, 1]),
    ('claim', 'sup_nosup', 'claim') : (sup_nosup_edges[:, 0], sup_nosup_edges[:, 1]),
})
graph

Graph(num_nodes={'claim': 16},
      num_edges={('claim', 'nosup_nosup', 'claim'): 2, ('claim', 'noun', 'claim'): 4, ('claim', 'sup_nosup', 'claim'): 16, ('claim', 'sup_sup', 'claim'): 16},
      metagraph=[('claim', 'claim', 'nosup_nosup'), ('claim', 'claim', 'noun'), ('claim', 'claim', 'sup_nosup'), ('claim', 'claim', 'sup_sup')])